# Table Considerations

#### Table Usage

The reason we have user tables and temporary tables is clear in so much as user tables are a more permanent object (they remain until manually dropped), whereas a temporary table remains only until the connection that created it is closed.

However, there is more to consider than that.

A user database, to name but a few, will be inside of a Disaster Recovery plan and therefore will be being backed up at regular intervals, will be maintained by overnight processes, requires database logs to be written in order to maintain database consistency, likely has strict permissions, and may have disk constraints which mean that it is kept small and grown only when necessary in a controlled fashion.

If you were to create tables at will within a user database (and populate them as we rarely leave empty tables lying around) you could be causing issues with all of the above.  Creating and populating a new table means it being included in the backup and recovery plan, which means time considerations, Service Level Agreements which could be breached as a result of the increased data, excess logging occurring which slows down the database throughput, increased maintenance windows, and unexpected database growth.

In some cases a user table is genuinely required and this should therefore be added to the database with the approval of your Database Administrator.  This is simply to highlight that consideration needs to be made.

Temporary tables, however, are stored in TempDB.  TempDB has no Disaster Recovery plan as the database is destroyed and re-created from scratch each time the SQL Server is restarted.  It has lighter logging and no maintenance.  It can grow, but usually will be pre-sized accordingly and is usually on disks which allow for better performance.  Therefore in most cases a temporary table is a better choice if the data you are storing does not need to be held permanently.

#### Table DataTypes

Another point to note is that we have so far been relatively precise with our datatype choices when creating tables and when learning the SQL language there is an inclination not to be so precise but to simply make each column as large as it needs to be.

For example, using `varchar(max)` for string fields, `bigint` for integers, and `decimal(39, 19)` for decimals is quite a common theme amongst those learning the language as they figure that it's easier and won't cause them problems.

However, there is a hidden cost to this which your Database Administrator will not appreciate.

In simple terms, a column defined as INT will take 4 bytes on disk for each value entered, whereas a BIGINT will take 8 bytes.

This may not sound a lot in the grand scheme of things, but let's imagine our table has 50 columns in it, all of them are integers and it contains a billion rows (pretty common in large firms).

If we do the maths we come to the following conclusion:

Space taken for INT columns = 4 bytes \* 50 \* 1 billion = 200 GB Space taken for BIGINT columns = 8 bytes \* 50 \* 1 billion = 400 GB

Therefore just by getting our datatypes correct we have saved our company 200 GB of disk space (let alone the cost of SQL Server processing 200 GB more than needed when reading or maintaining the table).

These costs are even more extreme with decimals and obviously we would have more than 1 table in our database.

Varchar is an exception to rule as it will only use the space required to hold its contents (abc will use 3 bytes, abcd will use 4 etc.). However, using max can cause SQL Server to internally create the table in a different and non-performant manner, it can also allow poor data to be entered into our database.

A key part of databases is that they are the golden source of our data and therefore if we insist (possibly due to company requirements or application limits) that a person's name can only be 25 characters then that's all we should allow otherwise we could find poor or erroneous data being added to the database which bloats our tables and causes external applications to fail.

Therefore we need to ensure that our tables are defined correctly at all times as per our needs.